# Data import
Script used to read txt and xls files and convert to pickled data files later imported by MembraneAnalysis

In [1]:
import pandas as pd
import pickle
import glob

## Settings
- dataDir = directory file path where txt files are stored (if applicable)
- data_file = file path to excel file (if applicable)
- result_name = name of result file

In [8]:
dataDir = "path\to\folder\UF_EXPORT_Raw_200401" 
data_file = "path\to\UF-fullskaleanläggning_processignaler_jan2018-april2019.xlsx"
result_name = 'joined_data_180101-200401_1m'

## Import txt files

In [4]:
filenames = glob.glob(dataDir + '*.txt') 
count=1
signals = []
for file in filenames:
    print('Importing file ' + str(count) + ' out of ' + str(len(filenames)))
    sensor_data = pd.read_csv(file, delimiter='\t')
    col = sensor_data.index
    index = []
    value = []
    for ix in range(len(col)):
        index.append(col[:][ix][0])
        value.append(col[:][ix][1])
            
    key = {'JAN':'01', 'FEB':'02', 'MAR':'03', 'APR':'04', 'MAJ':'05', 'JUN':'06', 'JUL':'07', 'AUG':'08', 'SEP':'09', 'OKT':'10', 'NOV':'11', 'DEC':'12'}
    index_mod = []
    for date in index:
        for month, num in key.items():
            date = date.replace(month, num)
        index_mod.append(date)

    index_dt = pd.to_datetime(index_mod, dayfirst=True)
    signals.append(pd.DataFrame(data=value, index=index_dt, columns=[sensor_data.columns[0]]))
    count += 1

    
txt_data = pd.DataFrame()
for s in signals:
    txt_data = txt_data.join(s, how='outer')

## Import Excel-file

In [6]:
xl = pd.ExcelFile(data_file,engine="openpyxl")
sheet_names = xl.sheet_names
excel_data = pd.DataFrame()
for sheet in sheet_names[1:]:
    print('Importing: ' + sheet)
    df = pd.read_excel(xl, sheet_name=sheet)  # Import data file to pandas Dataframe
    df.index = df['Tid']
    df = df.drop(['Tid'], axis=1)
    excel_data = pd.concat([excel_data, df])

excel_data = excel_data.drop_duplicates()

Importing: Janurari 2018
Importing: Februari 2018
Importing: Mars 2018
Importing: April 2018
Importing: Maj 2018
Importing: Juni 2018
Importing: Juli 2018
Importing: Augusti 2018
Importing: September 2018
Importing: Oktober 2018
Importing: November 2018
Importing: December 2018
Importing: Januari 2019
Importing: Februari 2019
Importing: Mars 2019
Importing: April 2019


## Merge and store data

In [9]:
joined_data = pd.concat([excel_data, txt_data])
joined_data_resampled = joined_data.resample('1T').median().ffill()
joined_data_resampled.to_pickle(result_name + '.pkl')